In [1]:
import pandas as pd
import numpy as np


In [2]:
data=pd.read_csv("D:\\New folder\\train.csv")

In [3]:
data.head

<bound method NDFrame.head of                           qid  \
0        00002165364db923c7e6   
1        000032939017120e6e44   
2        0000412ca6e4628ce2cf   
3        000042bf85aa498cd78e   
4        0000455dfa3e01eae3af   
...                       ...   
1306117  ffffcc4e2331aaf1e41e   
1306118  ffffd431801e5a2f4861   
1306119  ffffd48fb36b63db010c   
1306120  ffffec519fa37cf60c78   
1306121  ffffed09fedb5088744a   

                                             question_text  target  
0        How did Quebec nationalists see their province...       0  
1        Do you have an adopted dog, how would you enco...       0  
2        Why does velocity affect time? Does velocity a...       0  
3        How did Otto von Guericke used the Magdeburg h...       0  
4        Can I convert montra helicon D to a mountain b...       0  
...                                                    ...     ...  
1306117  What other technical skills do you need as a c...       0  
1306118  Does MS in E

In [4]:
data.drop("qid",axis=1,inplace=True)

In [5]:
data.head

<bound method NDFrame.head of                                              question_text  target
0        How did Quebec nationalists see their province...       0
1        Do you have an adopted dog, how would you enco...       0
2        Why does velocity affect time? Does velocity a...       0
3        How did Otto von Guericke used the Magdeburg h...       0
4        Can I convert montra helicon D to a mountain b...       0
...                                                    ...     ...
1306117  What other technical skills do you need as a c...       0
1306118  Does MS in ECE have good job prospects in USA ...       0
1306119                          Is foam insulation toxic?       0
1306120  How can one start a research project based on ...       0
1306121  Who wins in a battle between a Wolverine and a...       0

[1306122 rows x 2 columns]>

In [13]:
!wget --no-check-certificate -P ./data/ "http://nlp.stanford.edu/data/glove.6B.zip

--2021-05-06 20:36:07--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-05-06 20:36:08--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
  Self-signed certificate encountered.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-05-06 20:36:09--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/

In [14]:
import tensorflow as tf


 18900K .......... .......... .......... .......... ..........  2% 9.69M 2m47s
 18950K .......... .......... .......... .......... ..........  2% 35.0M 2m47s
 19000K .......... .......... .......... .......... ..........  2% 7.49M 2m47s
 19050K .......... .......... .......... .......... ..........  2% 18.9M 2m46s
 19100K .......... .......... .......... .......... ..........  2% 11.4M 2m46s
 19150K .......... .......... .......... .......... ..........  2% 36.6M 2m46s
 19200K .......... .......... .......... .......... ..........  2%  391K 2m51s
 19250K .......... .......... .......... .......... ..........  2% 11.6M 2m50s
 19300K .......... .......... .......... .......... ..........  2% 85.5M 2m50s
 19350K .......... .......... .......... .......... ..........  2% 3.90M 2m50s
 19400K .......... .......... .......... .......... ..........  2% 96.7M 2m50s
 19450K .......... .......... .......... .......... ..........  2% 8.33M 2m49s
 19500K .......... .......... .......... ..........

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
with tf.device('/device:GPU:0'):
    !unzip /content/glove.42B.300d.zip

In [ ]:
!rm /content/glove.42B.300d.zip

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train1,x_test1=train_test_split(data,test_size=0.35)

In [ ]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8',errors="ignore")

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [ ]:
x_train=x_train1['question_text']
y_train=x_train1['target']
x_test=x_test1['question_text']
y_test=x_test1['target']

In [ ]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model

In [ ]:
sent_lens=[len(word_tokenize(x)) for x in x_train]

In [ ]:
np.percentile(sent_lens,95)

In [ ]:
max_len=30

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [ ]:
embedding_matrix=np.zeros((vocab_size+1,300))
words_not_available=0
for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector
    else:
      words_not_available+=1

In [ ]:
words_not_available

In [ ]:
vocab_size

In [ ]:

inputs=Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer=LSTM(50)(embed)
dense1=Dense(10,activation='relu')(lstm_layer)
drop=Dropout(0.2)(dense1)
final_layer=Dense(1,activation='sigmoid')(drop)

model=Model(inputs=inputs,outputs=final_layer)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(seq_train_matrix,y_train.values,validation_data=(seq_test_matrix,y_test.values),epochs=40,batch_size=30000)

 49050K .......... .......... .......... .......... ..........  5% 42.9M 2m19s
 49100K .......... .......... .......... .......... ..........  5% 14.2M 2m19s
 49150K .......... .......... .......... .......... ..........  5% 71.4M 2m19s
 49200K .......... .......... .......... .......... ..........  5% 47.6M 2m19s
 49250K .......... .......... .......... .......... ..........  5% 15.6M 2m19s
 49300K .......... .......... .......... .......... ..........  5% 54.7M 2m19s
 49350K .......... .......... .......... .......... ..........  5% 6.68M 2m19s
 49400K .......... .......... .......... .......... ..........  5% 28.8M 2m19s
 49450K .......... .......... .......... .......... ..........  5% 77.3K 2m29s
 49500K .......... .......... .......... .......... ..........  5% 7.16M 2m29s
 49550K .......... .......... .......... .......... ..........  5% 3.71M 2m29s
 49600K .......... .......... .......... .......... ..........  5% 75.2M 2m29s
 49650K .......... .......... .......... .......... 